<a href="https://colab.research.google.com/github/adidror005/youtube-videos/blob/main/MultiLegOptionsYouTubeVideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup


### Install Alpaca-Py SDK

In [8]:
!pip install alpaca-py

### Grab API Key + API Secret

In [9]:
from google.colab import userdata
ALPACA_API_KEY=userdata.get('ALPACA_API_KEY')
ALPACA_SECRET_KEY=userdata.get('ALPACA_API_SECRET')

### Initialize Trading Client

In [11]:
from alpaca.trading.client import TradingClient
trading_client = TradingClient(ALPACA_API_KEY, ALPACA_SECRET_KEY,paper=True)

### Look at Open Orders

In [12]:
trading_client.get_orders()

[]

# Utility Function to Get Option Symbols

In [15]:
from alpaca.trading.requests import GetOptionContractsRequest
from alpaca.trading.enums import ContractType
def get_option_symbol(underlying_symbol, expiry, strike, option_type):
    option_contracts = trading_client.get_option_contracts(
        GetOptionContractsRequest(
            underlying_symbols=[underlying_symbol],
            expiration_date=expiry,
            strike_price_gte=str(strike),
            strike_price_lte=str(strike),
            type=option_type
        )
    ).option_contracts

    if len(option_contracts) != 1:
        raise ValueError("Did not find exactly 1 unique contract.")
    return option_contracts[0].symbol
get_option_symbol("AMD","2025-05-30",120,"call"),get_option_symbol("AMD","2025-06-06",100,"call")

('AMD250530C00120000', 'AMD250606C00100000')

### Define The Legs of the Contract (i.e. for Diagonal Call Spread)
* symbol
* side
* ratio_qty


In [21]:
from alpaca.trading.requests import OptionLegRequest
from alpaca.trading.enums import OrderSide

legs = [OptionLegRequest(symbol='AMD250606C00100000',
                         ratio_qty=1,side=OrderSide.BUY),
        OptionLegRequest(symbol='AMD250530C00120000',
                         ratio_qty=1,side=OrderSide.SELL)]



### Define Order Request and Submit Order
* order_class = OrderClass.MLEG
* legs = list[OptionLegRequest]
* Note: what is cool is we can define our own id via the *client_id* so we can manage IDs ourselves.


In [43]:
from alpaca.trading.requests import LimitOrderRequest, MarketOrderRequest
from alpaca.trading.enums import OrderSide, OrderClass, TimeInForce

limit_order_request = LimitOrderRequest(
    qty=1,
    order_class = OrderClass.MLEG,
    legs=legs,
    time_in_force=TimeInForce.DAY,
    limit_price = 5,
    )

order=trading_client.submit_order(
    order_data=limit_order_request
)

In [45]:
order.id

UUID('6397e245-2626-4ac5-a712-b2c4c1ec4494')

## Get Order By Id

In [48]:
trading_client.get_order_by_id(order.id)

{   'asset_class': None,
    'asset_id': None,
    'canceled_at': None,
    'client_order_id': 'f79a8a3e-eda8-4254-9f67-53c9219419ae',
    'created_at': datetime.datetime(2025, 5, 29, 16, 54, 53, 916329, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('6397e245-2626-4ac5-a712-b2c4c1ec4494'),
    'legs': [   {   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
                    'asset_id': UUID('cccb90a8-170b-447a-ac02-a5a1ef74c088'),
                    'canceled_at': None,
                    'client_order_id': 'a2a737fd-e3ce-4253-9857-1d50e1a14cb6',
                    'created_at': datetime.datetime(2025, 5, 29, 16, 54, 53, 916329, tzinfo=TzInfo(UTC)),
                    'expired_at': None,
                    'expires_at': datetime.datetime(2025, 5, 29, 20, 0, tzinfo=TzInfo(UTC)),
                 

### Get Order By Client Id

In [42]:
trading_client.get_order_by_client_id('test100')

{   'asset_class': None,
    'asset_id': None,
    'canceled_at': datetime.datetime(2025, 5, 29, 16, 53, 52, 656606, tzinfo=TzInfo(UTC)),
    'client_order_id': 'test100',
    'created_at': datetime.datetime(2025, 5, 29, 16, 53, 9, 538329, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('281b53d3-144d-4d1e-ae6f-8795d6e4a34c'),
    'legs': [   {   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
                    'asset_id': UUID('cccb90a8-170b-447a-ac02-a5a1ef74c088'),
                    'canceled_at': datetime.datetime(2025, 5, 29, 16, 53, 52, 656603, tzinfo=TzInfo(UTC)),
                    'client_order_id': 'c789a666-9433-46ba-a04d-f22e6a805359',
                    'created_at': datetime.datetime(2025, 5, 29, 16, 53, 9, 538329, tzinfo=TzInfo(UTC)),
                    'expired_at': None,
         

### Cancel Orders if Necessary

In [41]:
trading_client.cancel_order_by_id(order.id)

### Replace Order By ID
* Can only replace limit_price, qty--- Not individual symbols or ratios.

In [47]:
from alpaca.trading.requests import ReplaceOrderRequest
replaced_order=trading_client.replace_order_by_id(order.id,ReplaceOrderRequest(limit_price=20,qty=2))

In [52]:
replaced_order.id,replaced_order.replaces,order.id

(UUID('ff72110d-bab0-4544-b064-b8dfa63b5438'),
 UUID('6397e245-2626-4ac5-a712-b2c4c1ec4494'),
 UUID('6397e245-2626-4ac5-a712-b2c4c1ec4494'))

In [49]:
trading_client.get_order_by_id(replaced_order.id)

{   'asset_class': None,
    'asset_id': None,
    'canceled_at': None,
    'client_order_id': 'ffec686b-288d-44cc-8501-c3aa576c09e5',
    'created_at': datetime.datetime(2025, 5, 29, 16, 57, 51, 694313, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': datetime.datetime(2025, 5, 29, 16, 57, 51, 772112, tzinfo=TzInfo(UTC)),
    'filled_avg_price': '13.76',
    'filled_qty': '2',
    'hwm': None,
    'id': UUID('ff72110d-bab0-4544-b064-b8dfa63b5438'),
    'legs': [   {   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
                    'asset_id': UUID('cccb90a8-170b-447a-ac02-a5a1ef74c088'),
                    'canceled_at': None,
                    'client_order_id': '6f007834-75ea-4909-9431-5ddd91467d45',
                    'created_at': datetime.datetime(2025, 5, 29, 16, 57, 51, 694313, tzinfo=TzInfo(UTC)),
                    'expired_at': None,
                    'expires_at': datetim

### Good References

* Alpaca-Py Github: https://github.com/alpacahq/alpaca-py.git
* Multi-leg Options: https://github.com/alpacahq/alpaca-py/tree/master/examples/options
* My YouTube Channel: https://www.youtube.com/channel/UCZHN0IOGmmvY6JtquMoEn9w
  - Checkout Alpaca Playlist
  - Ultimate Getting Started with Alpaca Tutorial